## 1、加载并标准数据

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
batch_size=64
trainset=torchvision.datasets.CIFAR10(
    root='data',
    train=True,
    download=True,
    transform=transform
)
trainloader=torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True,num_workers=2)

testset=torchvision.datasets.CIFAR10(
    root='data',
    train=True,
    download=True,
    transform=transform
)
testloader=torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=False,num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
xtrain,ytrain=next(iter(trainloader))
xtest,ytest=next(iter(testloader))

In [5]:
xtrain.size(),ytrain.size()

(torch.Size([64, 3, 32, 32]), torch.Size([64]))

## 2、构建卷积神经网络

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(3,6,5)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(6,16,5)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)

    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [7]:
model=Net()
model

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

## 三、训练并测试神经网络

In [13]:
def train_loop(dataloader,model,optimizer,loss_fn):
    size=len(dataloader.dataset)
    for batch,(x,y) in enumerate(trainloader):
        optimizer.zero_grad()

        #forward
        prob=model(x)
        loss=loss_fn(prob,y)

        #backward
        loss.backward()
        
        #optimizer
        optimizer.step()

        #输出结果
        if batch%100 == 0:
            loss_value=loss.item()
            current=batch*len(y)
            print(f'Loss:{loss_value:.5f}  [{current:5d}/{size}]')

def test_loop(dataloader,model,optimizer,loss_fn):
    loss,correct=0,0
    batches=len(dataloader)
    size=len(dataloader.dataset)
    with torch.no_grad():
        for x,y in dataloader:
            prob=model(x)#64*10
            loss+=loss_fn(prob,y)

            _,pred=torch.max(prob,1)
            correct+=(pred==y).sum().item()
            
        error=loss/batches
        accuracy=correct/size
    print(f'测试集预测准确率为{accuracy:.3f},误差为{error:.3f}')


In [14]:
lr=1e-3
optimizer=torch.optim.SGD(model.parameters(),lr=lr,momentum=0.9)
loss_fn=nn.CrossEntropyLoss()

for i in range(1,6):
    print(f'第{i}次迭代\n---------')
    train_loop(trainloader,model,optimizer,loss_fn)
    test_loop(testloader,model,optimizer,loss_fn)

第1次迭代
---------
Loss:2.23625  [    0/50000]
Loss:2.25394  [ 6400/50000]
Loss:2.13815  [12800/50000]
Loss:2.02798  [19200/50000]
Loss:1.97634  [25600/50000]
Loss:1.92401  [32000/50000]
Loss:1.91915  [38400/50000]
Loss:1.96762  [44800/50000]
测试集预测准确率为0.307,误差为1.924
第2次迭代
---------
Loss:1.91362  [    0/50000]
Loss:1.85109  [ 6400/50000]
Loss:1.83916  [12800/50000]
Loss:1.95227  [19200/50000]
Loss:1.65078  [25600/50000]
Loss:2.05327  [32000/50000]
Loss:1.86030  [38400/50000]
Loss:1.61505  [44800/50000]
测试集预测准确率为0.364,误差为1.753
第3次迭代
---------
Loss:1.79240  [    0/50000]
Loss:1.57863  [ 6400/50000]
Loss:1.90574  [12800/50000]
Loss:1.66272  [19200/50000]
Loss:1.65947  [25600/50000]
Loss:1.56988  [32000/50000]
Loss:1.58912  [38400/50000]
Loss:1.68179  [44800/50000]
测试集预测准确率为0.423,误差为1.577
第4次迭代
---------
Loss:1.43490  [    0/50000]
Loss:1.49637  [ 6400/50000]
Loss:1.58993  [12800/50000]
Loss:1.41838  [19200/50000]
Loss:1.54780  [25600/50000]
Loss:1.62645  [32000/50000]
Loss:1.67458  [38400/500